# _**Baselines and Content-Based Recommender**_ #

In [2]:
import os
import math
import json
from collections import defaultdict, Counter
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import random
import warnings
warnings.filterwarnings('ignore')

In [3]:
movies = pd.read_csv("../data/final data/movies_metadata.csv")
ratings = pd.read_csv("../data/final data/ratings_small.csv")
links = pd.read_csv("../data/final data/links_small.csv")
credits = pd.read_csv("../data/final data/credits.csv")
keywords = pd.read_csv("../data/final data/keywords.csv")

In [4]:
print("Shapes:")
print("movies:", movies.shape)
print("ratings:", ratings.shape)
print("links:", links.shape)
print("keywords:", keywords.shape)
print("credits:", credits.shape)

Shapes:
movies: (45466, 27)
ratings: (100004, 4)
links: (9125, 3)
keywords: (46419, 3)
credits: (45476, 14)


In [5]:
print("movies columns:", movies.columns.tolist())
print("ratings columns:", ratings.columns.tolist())
print("links columns:", links.columns.tolist())
print("keywords columns:", keywords.columns.tolist())
print("credits columns:", credits.columns.tolist())

movies columns: ['adult', 'budget', 'id', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count', 'name_belongs_to_collection', 'id_belongs_to_collection', 'poster_path_belongs_to_collection', 'backdrop_path_belongs_to_collection', 'name_genres', 'id_genres', 'name_production_countries', 'iso_3166_1_production_companies', 'name_production_companies', 'id_production_companies']
ratings columns: ['userId', 'movieId', 'rating', 'timestamp']
links columns: ['movieId', 'imdbId', 'tmdbId']
keywords columns: ['id', 'name_keywords', 'id_keywords']
credits columns: ['id', 'name_crew', 'department_crew', 'gender_crew', 'job_crew', 'profile_path_crew', 'id_crew', 'name_cast', 'order_cast', 'gender_cast', 'credit_id_cast', 'profile_path_cast', 'id_cast', 'character_cast']


In [6]:
links = links.dropna(subset=["tmdbId", "movieId"])
links["tmdbId"] = pd.to_numeric(links["tmdbId"], errors="coerce")
links = links.dropna(subset=["tmdbId"]).astype({"tmdbId": int, "movieId": int})
ml_to_tmdb = dict(zip(links["movieId"], links["tmdbId"]))

In [7]:
print("original ratings rows:", len(ratings))
ratings = ratings[ratings["movieId"].isin(ml_to_tmdb.keys())].copy()
ratings["tmdbId"] = ratings["movieId"].map(ml_to_tmdb)
print("ratings after mapping:", len(ratings))

original ratings rows: 100004
ratings after mapping: 99933


In [8]:
ratings["timestamp"] = pd.to_datetime(ratings["timestamp"], unit="s")
ratings = ratings.sort_values(["userId", "timestamp"])
last_idx = ratings.groupby("userId")["timestamp"].idxmax()
test_df = ratings.loc[last_idx].copy()
train_df = ratings.drop(index=last_idx).copy()

In [9]:
min_train_interactions = 3
valid_users = train_df.groupby("userId").size()
valid_users = valid_users[valid_users >= min_train_interactions].index
train_df = train_df[train_df["userId"].isin(valid_users)]
test_df = test_df[test_df["userId"].isin(valid_users)]

In [10]:
print("train users:", train_df["userId"].nunique())
print("test rows:", len(test_df))

train users: 671
test rows: 671


WR (popularity baseline)

In [11]:
for col in ["vote_count", "vote_average", "id", "title"]:
    if col not in movies.columns:
        raise ValueError(f"movies missing column {col}")

movies_small = movies[["id", "title", "vote_count", "vote_average", "popularity"]].copy()
movies_small["vote_count"] = pd.to_numeric(movies_small["vote_count"], errors="coerce").fillna(0)
movies_small["vote_average"] = pd.to_numeric(movies_small["vote_average"], errors="coerce").fillna(0)

C = movies_small["vote_average"].mean()
m = movies_small["vote_count"].quantile(0.80)

def imdb_weighted_rating(row, m=m, C=C):
    v = row["vote_count"]
    R = row["vote_average"]
    return (v / (v + m)) * R + (m / (v + m)) * C if (v + m) > 0 else 0.0

movies_small["wr"] = movies_small.apply(imdb_weighted_rating, axis=1)

movies_small["id"] = pd.to_numeric(movies_small["id"], errors="coerce")

movies_small = movies_small.dropna(subset=["id"])

movies_small["id"] = movies_small["id"].astype(int)

In [12]:
candidate_tmdb = set(ml_to_tmdb.values())
pop_candidates = movies_small[movies_small["id"].isin(candidate_tmdb)].copy()
pop_candidates = pop_candidates.sort_values("wr", ascending=False)
pop_ranked_tmdb = pop_candidates["id"].tolist()
print("Candidates for popularity:", len(pop_ranked_tmdb))

Candidates for popularity: 9099


Content-Based Recommender (CB)

In [13]:
def to_tokens(s):
    if pd.isna(s) or s == "":
        return []
    return [t.strip().replace(" ", "_") for t in str(s).split(",") if t.strip()]

for df in [movies, keywords, credits]:
    df["id"] = pd.to_numeric(df["id"], errors="coerce")
    df.dropna(subset=["id"], inplace=True)
    df["id"] = df["id"].astype(int)

In [14]:
kw_map = keywords[keywords["id"].isin(candidate_tmdb)][["id", "name_keywords"]].set_index("id")["name_keywords"].to_dict()

cast_map = credits[credits["id"].isin(candidate_tmdb)][["id", "name_cast"]].set_index("id")["name_cast"].to_dict()

crew_map = credits[credits["id"].isin(candidate_tmdb)][["id", "name_crew"]].set_index("id")["name_crew"].to_dict()

movies_cb = movies[movies["id"].isin(candidate_tmdb)][["id","overview","tagline","name_genres","title"]].copy()

In [15]:
def build_item_text(row, kw_map=kw_map, cast_map=cast_map, crew_map=crew_map):
    parts = []
    if isinstance(row.get("overview"), str): parts.append(row["overview"])
    if isinstance(row.get("tagline"), str): parts.append(row["tagline"])
    # genres
    for tok in to_tokens(row.get("name_genres")):
        parts.append(f"GENRE_{tok}")
    # keywords
    kws = kw_map.get(row["id"])
    if isinstance(kws, str):
        for tok in to_tokens(kws):
            parts.append(f"KW_{tok}")
    # cast
    cast = cast_map.get(row["id"])
    if isinstance(cast, str):
        for tok in to_tokens(cast)[:10]:
            parts.append(f"CAST_{tok}")
    # crew
    crew = crew_map.get(row["id"])
    if isinstance(crew, str):
        for tok in to_tokens(crew)[:10]:
            parts.append(f"CREW_{tok}")
    return " ".join(parts)

In [17]:
movies_cb["cb_text"] = movies_cb.apply(build_item_text, axis=1)
print("CB items:", len(movies_cb))

CB items: 9099


TF-IDF

In [18]:
vectorizer = TfidfVectorizer(max_features=50000, min_df=2, ngram_range=(1,2))
X_items = vectorizer.fit_transform(movies_cb["cb_text"])

item_index = {tmdb_id: idx for idx, tmdb_id in enumerate(movies_cb["id"].tolist())}
index_item = {idx: tmdb_id for tmdb_id, idx in item_index.items()}

print("X_items shape:", X_items.shape)

X_items shape: (9099, 50000)


User profiles(cb)

In [19]:
train_tmdb = train_df[train_df["tmdbId"].isin(item_index.keys())].copy()
train_tmdb["item_idx"] = train_tmdb["tmdbId"].map(item_index)

user_ids = sorted(train_tmdb["userId"].unique())
uid_to_index = {u:i for i,u in enumerate(user_ids)}


train_user_mean = train_df.groupby("userId")["rating"].mean().to_dict()
train_tmdb["w"] = train_tmdb.apply(lambda r: r["rating"] - train_user_mean[r["userId"]], axis=1)

rows = train_tmdb["item_idx"].astype(int).to_numpy()
cols = train_tmdb["userId"].map(uid_to_index).astype(int).to_numpy()
data = train_tmdb["w"].astype(np.float32).to_numpy()

M = sparse.coo_matrix((data, (rows, cols)), shape=(X_items.shape[0], len(uid_to_index))).tocsr()

user_cb_profiles = M.T.dot(X_items)
user_cb_profiles = normalize(user_cb_profiles.tocsr(), norm="l2", axis=1)

print("user_cb_profiles shape:", user_cb_profiles.shape)

user_cb_profiles shape: (671, 50000)


In [20]:
def recommend_popular(k, exclude_set=None):
    rec = []
    for tmdb in pop_ranked_tmdb:
        if exclude_set and tmdb in exclude_set:
            continue
        rec.append(tmdb)
        if len(rec) >= k:
            break
    return rec

In [21]:
def recommend_cb(user_id, k=10, exclude_set=None, min_score=0.0):
    if user_id not in uid_to_index:
        return []
    u_idx = uid_to_index[user_id]
    uvec = user_cb_profiles[u_idx, :]
    if uvec.nnz == 0:
        return []
    sims = (uvec @ X_items.T).toarray().ravel()
    order = np.argsort(-sims)
    rec = []
    for iidx in order:
        tmdb = index_item.get(iidx)
        if tmdb is None:
            continue
        if exclude_set and tmdb in exclude_set:
            continue
        if sims[iidx] >= min_score:
            rec.append((tmdb, sims[iidx]))
        if len(rec) >= k:
            break
    return [t for t, s in rec]